In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline
import xgboost as xgb
pd.set_option('max_rows',100)

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
fixed=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/fixed_cost.csv')
dprice=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/demand_price.csv')
pdcost=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/production_cost.csv')
pdcap=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/production_capacity.csv')
dcost=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/delivery_cost.csv')
chdays=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/changeover_days.csv')
chcost=pd.read_csv('/Users/shashank/Downloads/av_or/train_HO825Pf/changeover_cost.csv')
forecast=pd.read_csv('/Users/shashank/Downloads/av_or/sample_submission_BqRDhl5/demand_forecast.csv')

In [16]:
dprice

,Product_ID,Region,Demand_price
0,P1,R1,0.000000
1,P2,R1,0.000000
2,P3,R1,0.000000
3,P4,R1,0.000000
4,P5,R1,0.000000
5,P6,R1,0.000000
6,P7,R1,0.000000
7,P8,R1,0.000000
8,P9,R1,0.000000
9,P10,R1,0.000000


In [5]:
dprice[dprice.Product=='P13'][dprice.Line==1][pdcap.Plant=='A']

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Plant,Line,Product,Capacity
12,A,1,P13,125


In [17]:
dprice

,Product_ID,Region,Demand_price
0,P1,R1,0.000000
1,P2,R1,0.000000
2,P3,R1,0.000000
3,P4,R1,0.000000
4,P5,R1,0.000000
5,P6,R1,0.000000
6,P7,R1,0.000000
7,P8,R1,0.000000
8,P9,R1,0.000000
9,P10,R1,0.000000


In [15]:
sub

NameError: name 'sub' is not defined

In [40]:
forecast[forecast.Product_ID=='P17'][forecast.Region=='R6']

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Product_ID,Region,Month,Demand
474,P17,R6,37,40.344759
475,P17,R6,38,37.630549
476,P17,R6,39,39.485166


In [41]:
df=forecast[forecast.Month==37]
df.reset_index(inplace=True,drop=True)

In [42]:
df2=df.copy(deep=True)
df3=df.copy(deep=True)

In [43]:
df2['Month']=38
df3['Month']=39


In [44]:
d=pd.concat([df,df2,df3],axis=0)

In [45]:
len(d)

4374

In [47]:
re=[]
for ind,i in d.iterrows():
    if i['Product_ID']=='P17':
        re.append(i['Demand']*5)
    elif i['Product_ID']=='P27':
        re.append(i['Demand']*5)
    elif i['Product_ID']=='P29':
        re.append(i['Demand']/0.6)
    elif i['Product_ID']=='P47':
        re.append(0)
    elif i['Product_ID']=='P25':
        re.append(0)
    elif i['Product_ID']=='P31':
        re.append(0)
        print 'ye5'
    elif i['Product_ID']=='P60':
        re.append(0)
    elif i['Product_ID']=='P57':
        re.append(0)
    elif i['Product_ID']=='P64':
        re.append(0)
    elif i['Product_ID']=='P61' and i['Region']=='R8':
        re.append(i['Demand']*89)   
    else: 
        re.append(i['Demand'])


ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5
ye5


In [48]:
len(re)

4374

In [49]:
d['Demand']=pd.Series(re)

In [50]:
d=d.sort_values(['Product_ID','Region','Month'])

In [51]:
seq=pd.read_csv('/Users/shashank/Downloads/seq.csv')

In [52]:
seq

,Plant,Line,Month,Day,Product_ID
0,A,1,37,1,P72
1,A,1,37,2,P72
2,A,1,37,3,P72
3,A,1,37,4,P72
4,A,1,37,5,NaN
5,A,1,37,6,NaN
6,A,1,37,7,P75
7,A,1,37,8,P75
8,A,1,37,9,NaN
9,A,1,37,10,NaN


In [53]:
seq[seq.Month==37][seq.Plant=='A'][seq.Line==1]['Product_ID'].value_counts()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


P72    4
P71    2
P75    2
P74    2
P58    2
P69    2
Name: Product_ID, dtype: int64

In [54]:
tran=pd.DataFrame(columns=['Product','Plant','Line','Produced'])

In [55]:
production=[]
for plant,line in [('A',1),('A',2),('B',1),('B',2),('B',3),('C',1)]:
    temp=seq[seq.Month==37][seq.Plant==plant][seq.Line==line]['Product_ID'].value_counts()
    for p in temp.index:
        cap=pdcap[pdcap.Product==p][pdcap.Plant==plant][pdcap.Line==line]['Capacity'].values[0]
        producing=cap*temp[p]
        production.append([p,plant,line,producing])
           

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [56]:
production=pd.DataFrame(production,columns=['Product','Plant','Line','Produced'])

In [57]:
regions=forecast[forecast.Product_ID=='P3'][forecast.Month==37].Region
regions=list(regions)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [58]:
main=pd.DataFrame()

In [59]:
for product in forecast.Product_ID.unique():
    temp=forecast[forecast.Product_ID==product][forecast.Month==37]
    ratio=temp['Demand']/temp['Demand'].sum()
    temp1=production[production.Product==product]
    fromA=0
    fromB=0
    fromC=0
    if len(temp1[temp1.Plant=='A']['Produced'])>0:
        fromA=temp1[temp1.Plant=='A']['Produced'].values[0]
    if len(temp1[temp1.Plant=='B']['Produced'])>0:
        fromB=temp1[temp1.Plant=='B']['Produced'].values[0]
    if len(temp1[temp1.Plant=='C']['Produced'])>0:
        fromC=temp1[temp1.Plant=='C']['Produced'].values[0]
    temp3=pd.DataFrame()
    if fromA>0:
        shipping=ratio*fromA
        temp3['Shipping_to_region_quantity']=shipping
        temp3['Plant']='A'
        temp3['Region']=regions
        temp3['Month']=37
        temp3['Product_ID']=product
    if fromB>0:
        shipping=ratio*fromB
        temp3['Shipping_to_region_quantity']=shipping
        temp3['Plant']='B'
        temp3['Region']=regions
        temp3['Month']=37
        temp3['Product_ID']=product
    if fromC>0:
        shipping=ratio*fromC
        temp3['Shipping_to_region_quantity']=shipping
        temp3['Plant']='C'
        temp3['Region']=regions
        temp3['Month']=37
        temp3['Product_ID']=product
    if len(temp3)>0: 
        print temp3
        main=main.append(temp3)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


     Shipping_to_region_quantity Plant Region  Month Product_ID
216                     0.000000     A     R1     37        P13
219                    34.603507     A    R10     37        P13
222                    23.104134     A    R11     37        P13
225                    19.404854     A    R12     37        P13
228                     0.000000     A    R13     37        P13
231                     0.000000     A    R14     37        P13
234                     0.000000     A    R15     37        P13
237                     0.000000     A    R16     37        P13
240                     0.000000     A    R17     37        P13
243                     8.742396     A    R18     37        P13
246                     0.000000     A     R2     37        P13
249                    18.455605     A     R3     37        P13
252                     0.000000     A     R4     37        P13
255                     0.000000     A     R5     37        P13
258                   275.814041     A  

      Shipping_to_region_quantity Plant Region  Month Product_ID
1512                     0.000000     B     R1     37        P35
1515                     0.000000     B    R10     37        P35
1518                   128.392729     B    R11     37        P35
1521                     0.000000     B    R12     37        P35
1524                     0.000000     B    R13     37        P35
1527                     0.000000     B    R14     37        P35
1530                     0.000000     B    R15     37        P35
1533                     0.000000     B    R16     37        P35
1536                     0.000000     B    R17     37        P35
1539                     5.426397     B    R18     37        P35
1542                     0.000000     B     R2     37        P35
1545                     0.000000     B     R3     37        P35
1548                     0.000000     B     R4     37        P35
1551                     0.000000     B     R5     37        P35
1554                     

      Shipping_to_region_quantity Plant Region  Month Product_ID
2862                     0.000000     A     R1     37        P58
2865                     7.603738     A    R10     37        P58
2868                    40.525431     A    R11     37        P58
2871                    24.473906     A    R12     37        P58
2874                     0.000000     A    R13     37        P58
2877                     0.000000     A    R14     37        P58
2880                     9.332513     A    R15     37        P58
2883                     0.000000     A    R16     37        P58
2886                     0.000000     A    R17     37        P58
2889                     6.181752     A    R18     37        P58
2892                     0.000000     A     R2     37        P58
2895                     0.000000     A     R3     37        P58
2898                     0.000000     A     R4     37        P58
2901                     0.000000     A     R5     37        P58
2904                    3

      Shipping_to_region_quantity Plant Region  Month Product_ID
3996                          0.0     B     R1     37        P77
3999                          0.0     B    R10     37        P77
4002                          0.0     B    R11     37        P77
4005                          0.0     B    R12     37        P77
4008                          0.0     B    R13     37        P77
4011                          0.0     B    R14     37        P77
4014                          0.0     B    R15     37        P77
4017                          0.0     B    R16     37        P77
4020                          0.0     B    R17     37        P77
4023                        252.0     B    R18     37        P77
4026                          0.0     B     R2     37        P77
4029                          0.0     B     R3     37        P77
4032                          0.0     B     R4     37        P77
4035                          0.0     B     R5     37        P77
4038                     

In [60]:
main.head()

,Shipping_to_region_quantity,Plant,Region,Month,Product_ID
216,0.000000,A,R1,37,P13
219,34.603507,A,R10,37,P13
222,23.104134,A,R11,37,P13
225,19.404854,A,R12,37,P13
228,0.000000,A,R13,37,P13


In [61]:
print np.sum(production[production.Product=='P35']['Produced'])
print np.sum(production[production.Product=='P37']['Produced'])

1260
560


In [62]:
t1=forecast[forecast.Product_ID=='P35'][forecast.Month==37]
t2=forecast[forecast.Product_ID=='P37'][forecast.Month==37]
r1=t1['Demand']/t1['Demand'].sum()
r2=t2['Demand']/t2['Demand'].sum()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [63]:
sh1=r1*1260
sh2=r2*560

In [64]:
p1=pd.DataFrame({'Shipping_to_region_quantity':sh1,'Plant':'B','Region':regions,'Month':37,'Product_ID':'P35'})
p2=pd.DataFrame({'Shipping_to_region_quantity':sh2,'Plant':'B','Region':regions,'Month':37,'Product_ID':'P37'})

In [65]:
main=main.append(p1)
main=main.append(p2)

In [66]:
len(main)/18

40

In [67]:
main

,Month,Plant,Product_ID,Region,Shipping_to_region_quantity
216,37,A,P13,R1,0.000000
219,37,A,P13,R10,34.603507
222,37,A,P13,R11,23.104134
225,37,A,P13,R12,19.404854
228,37,A,P13,R13,0.000000
231,37,A,P13,R14,0.000000
234,37,A,P13,R15,0.000000
237,37,A,P13,R16,0.000000
240,37,A,P13,R17,0.000000
243,37,A,P13,R18,8.742396


In [68]:
main.reset_index(inplace=True,drop=True)

In [69]:
cols=['Plant','Region','Product_ID','Month','Shipping_to_region_quantity']

In [70]:
main=main[cols]

In [71]:
forecast[forecast.Product_ID=='P13'][forecast.Month==37][forecast.Region=='R13']

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Product_ID,Region,Month,Demand
228,P13,R13,37,0.0


In [72]:
len(main)

720

In [73]:
df=main.copy(deep=True)
o2=df.copy(deep=True)
o3=df.copy(deep=True)
o2['Month']=38
o3['Month']=39
df=df.append(o2)
df=df.append(o3)

In [74]:
df

,Plant,Region,Product_ID,Month,Shipping_to_region_quantity
0,A,R1,P13,37,0.000000
1,A,R10,P13,37,34.603507
2,A,R11,P13,37,23.104134
3,A,R12,P13,37,19.404854
4,A,R13,P13,37,0.000000
5,A,R14,P13,37,0.000000
6,A,R15,P13,37,0.000000
7,A,R16,P13,37,0.000000
8,A,R17,P13,37,0.000000
9,A,R18,P13,37,8.742396


In [75]:
temp=pd.read_csv('/Users/shashank/Downloads/av_or/sample_submission_BqRDhl5/shipping_region.csv')

In [76]:
df=df.rename(columns={'Shipping_to_region_quantity':'Shipping_to_region_quantity'})

In [77]:
temp.drop(['Shipping_to_region_quantity'],axis=1,inplace=True)

In [78]:
temp1=temp.merge(df,on=['Plant','Region','Product_ID','Month'],how='left')

In [79]:
temp1.fillna(-2,inplace=True)

In [80]:
(temp1[temp1.Shipping_to_region_quantity!=-2])['Shipping_to_region_quantity'].sum()

50582.99999999998

In [81]:
df.Shipping_to_region_quantity.sum()

50583.0

In [85]:
m=temp1.replace(-2,0)

In [86]:
k=m.drop_duplicates(subset=['Plant','Region','Product_ID','Month'])

In [87]:
k['Shipping_to_region_quantity']=np.floor(k.Shipping_to_region_quantity)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [592]:
k.to_csv('/Users/shashank/Desktop/shipping_region.csv',index=False)

In [593]:
d.to_csv('/Users/shashank/Desktop/demand_forecast.csv',index=False)

In [594]:
seq.to_csv('/Users/shashank/Desktop/manufacture_sequence.csv',index=False)

In [89]:
seq

,Plant,Line,Month,Day,Product_ID
0,A,1,37,1,P72
1,A,1,37,2,P72
2,A,1,37,3,P72
3,A,1,37,4,P72
4,A,1,37,5,NaN
5,A,1,37,6,NaN
6,A,1,37,7,P75
7,A,1,37,8,P75
8,A,1,37,9,NaN
9,A,1,37,10,NaN


In [596]:
len(temp)

13122

In [518]:
# p33 m37

In [523]:
d

,Product_ID,Region,Month,Demand
0,P1,R1,37,0.0
0,P1,R1,38,0.0
0,P1,R1,39,0.0
1,P1,R10,37,0.0
1,P1,R10,38,0.0
1,P1,R10,39,0.0
2,P1,R11,37,0.0
2,P1,R11,38,0.0
2,P1,R11,39,0.0
3,P1,R12,37,0.0
